<a href="https://colab.research.google.com/github/Stephenwang3801/APS360-Project/blob/main/LargerGeneratePoemsFromAmazonReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')
home_directory = '/content/drive/MyDrive/Aps360 Project/'
model_name = 'model_amazon_review_poem_part2'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 7.7 MB/s 
     |████████████████████████████████| 895 kB 43.5 MB/s 
     |████████████████████████████████| 636 kB 43.9 MB/s 
     |████████████████████████████████| 3.3 MB 62.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import numpy as np
import pandas as pd 

import random
import time
import datetime
import gc

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler, Subset

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

special_tokens_dict = {'bos_token': '<BOS>', 'sep_token':'<SEP>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [5]:
RANDOM_SEED = 73
BATCH_SIZE = 5
EPOCHS = 8
MAX_LEN = 400

In [6]:
class PoemWithSummaryDataset(Dataset):
    
    def __init__(self, input, target, tokenizer, gpt2_type='gpt2', max_length=MAX_LEN):
        self.tokenizer = tokenizer
        self.data = []
        self.attn_masks = []
        
        for i in range(len(input)):
          
          
          encodings_dict = tokenizer(str(input[i]) + '<SEP>' + str(target[i]),
                                    truncation=True,
                                    max_length=max_length,
                                    padding='max_length'
                                  )
          """
          #For checking how long the data is
          encodings_dict = tokenizer(input[i] + '<SEP>' + target[i]) 
          """
          
          self.data.append(torch.tensor(encodings_dict['input_ids']))
          self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        
        return len(self.data)
    
    def __getitem__(self, idx):
        
        return self.data[idx], self.attn_masks[idx]
        

In [7]:
#Helper functions
def get_train_val_size(split, dataset):
    train_size = int(split * len(dataset))
    val_size = len(dataset) - train_size
    return train_size, val_size
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [8]:
def train(poem_model,learning_rate=1e-4,eps=1e-8,warmup_steps=50,starting_epoch=0):
  optimizer = AdamW(poem_model.parameters(), lr=learning_rate, eps=eps)
  total_steps = len(poem_train_dataloader) * EPOCHS
  scheduler = get_linear_schedule_with_warmup(optimizer,
                                              num_warmup_steps=warmup_steps,
                                              num_training_steps=total_steps)
  start_time = time.time()
  train_loss = []
  val_loss = []
  for epoch_i in range(starting_epoch, EPOCHS):
      print(f'Epoch {epoch_i + 1} of {EPOCHS}')
      t0 = time.time()

      #Train the model
      total_train_loss = 0
      poem_model.train()
      for step, batch in enumerate(poem_train_dataloader):

          #Note that the labels are the same as the input. This is because the 
          #GPT2LMHeadModel That we are using shifts the labels by 1 meaning that 
          #the label for each input token is the next input token. This is desired 
          #when building a language model because we want the predicted output to
          #be the next most likely word in the sentence. 
          b_input_ids = batch[0].to(device)
          b_masks = batch[1].to(device)

          poem_model.zero_grad()        
          outputs = poem_model(b_input_ids,
                                      labels=b_input_ids,
                                      attention_mask=b_masks,
                                      token_type_ids=None)
          loss = outputs[0]  
          batch_loss = loss.item()
          total_train_loss += batch_loss

          loss.backward()
          optimizer.step()
          scheduler.step()

      avg_train_loss = total_train_loss / len(poem_train_dataloader)       
      training_time = format_time(time.time() - t0)
      print(f'Average Training Loss: {avg_train_loss}. Epoch Training Time: {training_time}')
      
      t0 = time.time()

      #Evaluate the model
      poem_model.eval()
      total_eval_loss = 0
      nb_eval_steps = 0
      for batch in poem_val_dataloader:
          b_input_ids = batch[0].to(device)
          b_masks = batch[1].to(device)

          with torch.no_grad():        
              outputs  = poem_model(b_input_ids,
                                          attention_mask=b_masks,
                                          labels=b_input_ids)
              loss = outputs[0]  

          batch_loss = loss.item()
          total_eval_loss += batch_loss        

      avg_val_loss = total_eval_loss / len(poem_val_dataloader)

      train_loss.append(avg_train_loss)
      val_loss.append(avg_val_loss)
      torch.save(poem_model.state_dict(), "{}/{}".format(
              home_directory,model_name))
      print(f'Average Validation Loss: {avg_val_loss}')

  np.savetxt("{}/{}_train_loss.csv".format(home_directory, model_name), train_loss)
  np.savetxt("{}/{}_val_loss.csv".format(home_directory, model_name), val_loss)
  print(f'Total Training Time: {format_time(time.time()-start_time)}')


##Loading the data

In [11]:
#Load them poems we want to train our model with
#poem_df = pd.read_csv(home_directory + 'poe_poems_with_summary_and_title.csv')
#poem_df = pd.read_csv(home_directory + 'InshortsCleanedData.csv')
#poem_df = pd.read_csv(home_directory + 'Datasets/modified_AmazonFoodReviews_10000.csv')
poem_df = pd.read_csv(home_directory + 'Poem Datasets/poetryFoundation/kaggle_poem_dataset_10000.csv')

inputs = poem_df['Title']
targets = poem_df['Content']
#inputs = poem_df['Summary']
#targets = poem_df['Review']

inputs = inputs.values
targets = targets.values


#Load the GPT2 tokenizer that will be used by PoemDataset to encode the poems. Add the 
#BOS,EOS and PAD tokens to the tokenized dictionary so that when we put these 
#Tokens around our poems to separate them, the tokenizer will know what to do with them.
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'sep_token':'<SEP>', 'pad_token': '<PAD>'}
tokenizer.add_special_tokens(special_tokens_dict)

#create an object of the PoemWithSummaryDataset class
#that will hold an ordered list of the tokenized version of each poem and its summary
poem_dataset = PoemWithSummaryDataset(inputs, targets, tokenizer)



In [12]:
#Split the poem dataset into a training set and a validation set.
poem_train_size, poem_val_size = get_train_val_size(split=0.8, dataset=poem_dataset)

#FOR TESTING
"""
poem_train_size = 800
poem_val_size = 200
poem_train_dataset = Subset(poem_dataset,range(0,800))
poem_val_dataset = Subset(poem_dataset,range(800,1000))
"""
poem_train_dataset, poem_val_dataset = random_split(poem_dataset, [poem_train_size, poem_val_size])
poem_train_dataloader = DataLoader(poem_train_dataset,
                              sampler=RandomSampler(poem_train_dataset),
                              batch_size=BATCH_SIZE)
poem_val_dataloader = DataLoader(poem_val_dataset,
                            sampler=SequentialSampler(poem_val_dataset),
                            batch_size=BATCH_SIZE)

In [13]:
"""
#checking how large the encodings should be
num_over = 0
total = len(poem_dataset.data)
for target in poem_dataset.data:
  if len(target) > 200:
    num_over+=1
print(num_over)
print(total)
"""

'\n#checking how large the encodings should be\nnum_over = 0\ntotal = len(poem_dataset.data)\nfor target in poem_dataset.data:\n  if len(target) > 200:\n    num_over+=1\nprint(num_over)\nprint(total)\n'

##Setup the model

In [14]:
torch.cuda.manual_seed_all(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device('cuda')

gc.collect()
torch.cuda.empty_cache() 

#If you are only evaluating the model and don't want to train it again,
#set training_desired to false. Otherwise set it to true and choose the 
#epoch you left off at last time to start Training from (0 if you haven't).
load_previous_state_dict = True
previous_state_dict_location = "model_amazon_review_poem_part2"#"model_summary_poe_epoch_7"


#Setup the pretrained GPT2 model
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions=MAX_LEN).from_pretrained('gpt2', output_hidden_states=True)
poem_model = GPT2LMHeadModel.from_pretrained('gpt2', config=configuration)
poem_model.resize_token_embeddings(len(tokenizer))

if load_previous_state_dict:
  poem_model.load_state_dict(torch.load("{}/{}".format(home_directory,previous_state_dict_location)))

poem_model.cuda()
poem_model = poem_model.to(device)

##Train the model if desired

In [15]:
!nvidia-smi

Tue Aug  3 01:51:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W /  70W |   1608MiB / 15109MiB |     31%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [17]:
training_desired = False

#If training_desired is set to true, choose the epoch you left off at last time
#to continue training from there (put it as 0 if you haven't yet)
starting_epoch = 0

# hyperparameters
learning_rate = 1e-4
eps = 1e-8
warmup_steps = 50
if training_desired:
  train(poem_model,learning_rate,eps,warmup_steps,starting_epoch)


##Use the model to generate poems

In [18]:
# create text generation seed prompt
prompts = ["Very satisfying drip coffee, tastes great black."]


In [19]:
poem_model.eval()
           
for prompt in prompts:
  encoded = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = encoded.to(device)
  sample_outputs = poem_model.generate(
                                  generated, 
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length=MAX_LEN,
                                  top_p=0.95, 
                                  num_return_sequences=10
                                  )
  print("\nPrompt: ", prompt, "\n Generated:\n")

  for i, sample_output in enumerate(sample_outputs):
      print("{}: {}\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
  

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Prompt:  Very satisfying drip coffee, tastes great black. 
 Generated:

0: Very satisfying drip coffee, tastes great black.I wake up feeling like I know I’m lost, but still,
I have a feeling I’m coming to blows,
though the way my eyes and my mouth
rejoist a kind of clarity,
the way a man can think like he has thought,
the way a person can get a man to the bottom
of a mountain or a lake.I wake up feeling like I know I’m lost,
but still, I feel like I’m breathing too much,
and I know, with great care, I’m breathing too much.
All the rest
I have, although I have lost it to my body,
to my feelings.

Sometimes I feel like I’m not real,
like I’m too good,
like I’m not really real.
But I’m more real, like I’m real
and real enough to feel really, real
like I’m real.

And I want to know how to go about
the whole thing,

the whole place,
the whole time.I wake up feeling like I know I’m lost,
but still, I feel like I’m breathing too much,
and I’m breathing too much,
and I feel, with great care, 